# About

Training of LuminarSequenceClassifier on the PrismAI dataset.

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
import sys
import torch

from typing import Final, Callable
from pathlib import Path

sys.path.insert(0, str(Path().resolve().parent.parent))

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, ConcatDataset
from IPython.display import display, Markdown
from datasets import load_dataset
from numpy._typing import NDArray
from torch.utils.data import Dataset, Subset
from datasets import DatasetDict
from src.luminar.utils.data import get_pad_to_fixed_length_fn, get_matched_datasets
from src.luminar.utils.training import ConvolutionalLayerSpec
from src.luminar.encoder import LuminarEncoder

import numpy as np
import glob

In [16]:
class Config:
    HF_TOKEN: Final[str] = (Path.home() / ".hf_token").read_text().strip()
    #DATASET_PATH: Final[str] = "liberi-luminaris/PrismAI-encoded-gpt2"
    DATASET_ROOT_PATH: Final[str] = "/storage/projects/stoeckel/prismai/encoded/fulltext/"
    #DATASET_ROOT_PATH: Final[str] = "/mnt/c/home/projects/prismAI/data/encoded/fulltext/"
    NUM_INTERMEDIATE_LIKELIHOODS: Final[int] = 13
    FEATURE_LEN = 512
    SEED = 42


## Loading & Preprocessing the datasets

Load the datasets for the training.

In [17]:
# We need that to sentence tokenize the text
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /home/staff_homes/kboenisc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [131]:
def sentence_to_token_spans(text: str, span_min_length : int = -1) -> list[tuple[int, int]]:
    """Return a list of (start_token_idx, end_token_idx) for each sentence in the text."""
    sentences = sent_tokenize(text)
    spans = []
    current_token_idx = 0

    for sent in sentences:
        tokens = luminar_encoder.tokenize(sent)["input_ids"]
        token_count = len(tokens)

        start = current_token_idx
        end = min(current_token_idx + token_count, Config.FEATURE_LEN)

        if start >= Config.FEATURE_LEN:
            break  # Stop if padding region or overflow
        # Testing: only take spans of a minimal length
        if end - start > span_min_length:
            spans.append((start, end))
        current_token_idx = end

    return spans


In [208]:
domains = ['bundestag', 'student_essays']
agents = ['gpt_4o_mini_gemma2_9b']
feature_agents = ['gpt2_512']

luminar_encoder = LuminarEncoder(max_len=Config.FEATURE_LEN)

In [209]:
datasets = {}

pad_to_fixed_length: Callable[[NDArray], NDArray] = get_pad_to_fixed_length_fn(Config.FEATURE_LEN)

for domain in domains:
    for agent in agents:
        for feature_agent in feature_agents:
            dataset_path = Path(Config.DATASET_ROOT_PATH) / agent / feature_agent / domain

            if not dataset_path.exists():
                raise FileNotFoundError(f"Dataset path {dataset_path} does not exist.")

            data_files = {
                "train": sorted(str(f) for f in dataset_path.glob("train/*.arrow")),
                "test": sorted(str(f) for f in dataset_path.glob("test/*.arrow")),
                "eval": sorted(str(f) for f in dataset_path.glob("eval/*.arrow")),
            }
            data_files = {k: v for k, v in data_files.items() if v}

            # These datasets are already matched, so we can load them directly
            # We need the tokenized text since we label sequences based on sentences.
            dataset_dict = (
                load_dataset(
                    "arrow",
                    data_files=data_files,
                )
                .map(
                    lambda batch: {
                        "tokenized_text": [
                            pad_to_fixed_length(
                                np.array(luminar_encoder.tokenize(t)["input_ids"]).reshape(-1, 1)
                            )
                            for t in batch["text"]
                        ],
                        "sentence_token_spans": [
                            sentence_to_token_spans(t)
                            for t in batch["text"]
                        ],
                    },
                    batched=True,
                    desc="Tokenizing, padding, and aligning sentences"
                )
                .map(
                    lambda batch: {
                        "span_labels": [
                            [label] * len(spans)
                            for label, spans in zip(batch["labels"], batch["sentence_token_spans"])
                        ]
                    },
                    batched=True,
                    desc="Assigning labels to sentence spans"
                )
            )

            datasets.setdefault(domain, {}).setdefault(agent, {})[feature_agent] = dataset_dict
            print(f"Loaded dataset for domain '{domain}' with agent '{agent}' and feature agent '{feature_agent}'")

datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Tokenizing, padding, and aligning sentences:   0%|          | 0/14078 [00:00<?, ? examples/s]

Tokenizing, padding, and aligning sentences:   0%|          | 0/4024 [00:00<?, ? examples/s]

...

Assigning labels to sentence spans:   0%|          | 0/50734 [00:00<?, ? examples/s]

Assigning labels to sentence spans:   0%|          | 0/14496 [00:00<?, ? examples/s]

Assigning labels to sentence spans:   0%|          | 0/7248 [00:00<?, ? examples/s]

Loaded dataset for domain 'student_essays' with agent 'gpt_4o_mini_gemma2_9b' and feature agent 'gpt2_512'


{'bundestag': {'gpt_4o_mini_gemma2_9b': {'gpt2_512': DatasetDict({
       train: Dataset({
           features: ['agent', 'id_sample', 'id_source', 'labels', 'text', 'features', 'tokenized_text', 'sentence_token_spans', 'span_labels'],
           num_rows: 14078
       })
       test: Dataset({
           features: ['agent', 'id_sample', 'id_source', 'labels', 'text', 'features', 'tokenized_text', 'sentence_token_spans', 'span_labels'],
           num_rows: 4024
       })
       eval: Dataset({
           features: ['agent', 'id_sample', 'id_source', 'labels', 'text', 'features', 'tokenized_text', 'sentence_token_spans', 'span_labels'],
           num_rows: 2012
       })
   })}},
 'student_essays': {'gpt_4o_mini_gemma2_9b': {'gpt2_512': DatasetDict({
       train: Dataset({
           features: ['agent', 'id_sample', 'id_source', 'labels', 'text', 'features', 'tokenized_text', 'sentence_token_spans', 'span_labels'],
           num_rows: 50734
       })
       test: Dataset({
         

In [211]:
# Sanity check
idx = 0
for domain, agents_dict in datasets.items():
    for agent, feature_agents_dict in agents_dict.items():
        for feature_agent, dataset in feature_agents_dict.items():
            md = f"""
**Domain:** `{domain}`
**Agent:** `{agent}`
**Feature Agent:** `{feature_agent}`

**Train:** {len(dataset['train'])}
**Test:** {len(dataset['test'])}
**Eval:** {len(dataset['eval'])}

**Example-Features:**
`{dataset['train'][idx]['features'][:2]}...`

**Feature-Shape:**
`{np.asarray(dataset['train'][idx]['features']).shape}`

**Example text:**
`{dataset['train'][idx]['text']}`

**Example-Tokenized Text:**
`{dataset['train'][idx]['tokenized_text'][:10]}...`

**Tokenized Text Shape:**
`{np.asarray(dataset['train'][idx]['tokenized_text']).shape}`

**Sentence-Token-Spans:**
`{dataset['train'][idx]['sentence_token_spans']}`

**Example Sentence-Token Span decoded:**
`{luminar_encoder.tokenizer.decode(np.asarray(dataset['train'][idx]['tokenized_text'])[dataset['train'][idx]['sentence_token_spans'][0][0]:dataset['train'][idx]['sentence_token_spans'][0][1]].flatten().tolist())}`

**Example span labels:**
`{dataset['train'][idx]['span_labels']}`

**Example label:**
`{dataset['train'][idx]['labels']}`

---
"""
            display(Markdown(md))


**Domain:** `bundestag`
**Agent:** `gpt_4o_mini_gemma2_9b`
**Feature Agent:** `gpt2_512`

**Train:** 14078
**Test:** 4024
**Eval:** 2012

**Example-Features:**
`[[1.8125965652870946e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00021406936866696924], [1.058023485711601e-06, 6.257003803966654e-08, 1.0760977176005326e-07, 1.8288755398998546e-08, 5.383418155702202e-10, 4.788487661944174e-11, 1.5573458321538336e-13, 2.799126584192487e-17, 1.2352709805983406e-22, 1.2212475851755014e-28, 6.798245356840774e-40, 0.0, 0.00011042043479392305]]...`

**Feature-Shape:**
`(512, 13)`

**Example text:**
`Sehr geehrte Damen und Herren, heute stehen wir hier im Deutschen Bundestag, um über ein Thema zu diskutieren, das für unsere Gesellschaft von entscheidender Bedeutung ist: die Gesundheitspolitik in Deutschland. Die Herausforderungen, vor denen unser Gesundheitssystem steht, sind nicht nur vielschichtig, sondern auch von drängender Aktualität, insbesondere im Kontext der COVID-19-Pandemie und ihrer weitreichenden Folgen. Es ist an der Zeit, dass wir uns den Fragen der Finanzierung und der strukturellen Reformen stellen, die notwendig sind, um ein Gesundheitssystem zu gewährleisten, das allen Bürgerinnen und Bürgern zugutekommt. Die Pandemie hat nicht nur die Schwächen unseres Gesundheitssystems offengelegt, sondern auch die Dringlichkeit von Reformen unterstrichen. Wir haben erlebt, wie überlastet unsere Krankenhäuser waren, wie das Pflegepersonal am Limit seiner Belastbarkeit operierte und wie die fehlende Ausstattung in vielen Einrichtungen zu einem echten Problem wurde. Diese Herausforderungen sind nicht neu, sie sind jedoch durch die Pandemie in den Fokus gerückt worden. Wir können nicht länger ignorieren, dass wir in der Vergangenheit versäumt haben, in die Zukunft unserer Gesundheitsversorgung zu investieren. Ein zentraler Punkt in dieser Debatte ist die Finanzierung des Gesundheitssystems. Der Gesundheitsfonds, der ursprünglich geschaffen wurde, um die Finanzierung der gesetzlichen Krankenversicherung zu stabilisieren, steht heute vor ernsthaften Herausforderungen. Die steigenden Kosten im Gesundheitswesen, die durch die COVID-19-Pandemie noch verstärkt wurden, erfordern ein Umdenken in der Finanzierungsstrategie. Wir müssen sicherstellen, dass die Mittel dort ankommen, wo sie am dringendsten benötigt werden – in den Kliniken, bei den Pflegekräften und in der Prävention. Die Bundesregierung hat in der Vergangenheit zahlreiche Maßnahmen ergriffen, um die finanziellen Belastungen während der Pandemie abzufedern. Doch ich frage Sie: Reichen diese Maßnahmen aus? Sind wir bereit, die Lehren aus dieser Krise zu ziehen und die notwendigen Schritte zu unternehmen, um unser Gesundheitssystem nachhaltig zu stärken? Ich bin der Überzeugung, dass wir nicht nur kurzfristige Lösungen suchen dürfen, sondern auch langfristige Strategien entwickeln müssen, die die strukturellen Probleme angehen. Ein weiterer Aspekt, den wir nicht außer Acht lassen dürfen, ist die Digitalisierung im Gesundheitswesen. Die Pandemie hat uns gezeigt, wie wichtig digitale Lösungen sind, um die Patientenversorgung zu verbessern und den Austausch von Informationen zu erleichtern. Dennoch stehen wir vor der Herausforderung, dass viele Einrichtungen nicht ausreichend digitalisiert sind. Hier müssen wir investieren, um die digitale Infrastruktur auszubauen und sicherzustellen, dass alle Bürgerinnen und Bürger von den Vorteilen der Digitalisierung profitieren können. Es ist auch unerlässlich, dass wir die Rolle der Pflegekräfte in unserer Gesellschaft neu bewerten. Die Pandemie hat uns eindringlich vor Augen geführt, wie wichtig das Pflegepersonal für die Funktionsfähigkeit unseres Gesundheitssystems ist. Dennoch sehen wir, dass die Arbeitsbedingungen und die Bezahlung in vielen Bereichen unzureichend sind. Wir müssen die Wertschätzung für diese Berufe erhöhen und die Rahmenbedingungen so gestalten, dass Pflegekräfte ihre wichtige Arbeit unter würdigen Bedingungen leisten können. Darüber hinaus müssen wir auch die Prävention in den Fokus rücken. Die COVID-19-Pandemie hat uns gelehrt, wie wichtig es ist, frühzeitig zu handeln, um Gesundheitsrisiken zu minimieren. Investitionen in präventive Maßnahmen sind nicht nur eine Frage der Gerechtigkeit, sondern auch eine ökonomische Notwendigkeit. Wir sollten uns nicht nur auf die Behandlung von Krankheiten konzentrieren, sondern auch auf deren Verhinderung. Hierzu gehört auch die Förderung von gesundheitsbewusstem Verhalten in der Bevölkerung und der Zugang zu Informationen über gesunde Lebensweisen. Ich möchte auch die Bedeutung der Zusammenarbeit zwischen Bund, Ländern und Kommunen betonen. Nur wenn wir gemeinsam an einem Strang ziehen, können wir die Herausforderungen, vor denen unser Gesundheitssystem steht, bewältigen. Es ist wichtig, dass wir die verschiedenen Akteure im Gesundheitswesen – von den Ärzten über die Krankenhäuser bis hin zu den Pflegeeinrichtungen – in den Reformprozess einbeziehen. Nur so können wir sicherstellen, dass die Lösungen, die wir entwickeln, auch tatsächlich den Bedürfnissen der Menschen entsprechen. Abschließend möchte ich betonen, dass wir jetzt die Chance haben, die Weichen für die Zukunft unseres Gesundheitssystems zu stellen. Lassen Sie uns diese Gelegenheit nutzen, um die notwendigen Reformen einzuleiten und ein Gesundheitssystem zu schaffen, das nicht nur auf Krisen reagiert, sondern proaktiv handelt. Wir müssen die Herausforderungen annehmen und gemeinsam für ein Gesundheitssystem kämpfen, das allen Bürgerinnen und Bürgern gerecht wird. Ich fordere Sie alle auf, sich an dieser wichtigen Debatte zu beteiligen. Lassen Sie uns gemeinsam für eine bessere Gesundheitspolitik in Deutschland eintreten, die den Menschen in den Mittelpunkt stellt und die Herausforderungen der Zukunft aktiv angeht. Vielen Dank für Ihre Aufmerksamkeit.`

**Example-Tokenized Text:**
`[[4653], [11840], [308], [1453], [11840], [660], [5245], [268], [3318], [2332]]...`

**Tokenized Text Shape:**
`(512, 1)`

**Sentence-Token-Spans:**
`[[0, 78], [78, 158], [158, 237], [237, 283], [283, 360], [360, 401], [401, 455], [455, 484], [484, 512]]`

**Example Sentence-Token Span decoded:**
`Sehr geehrte Damen und Herren, heute stehen wir hier im Deutschen Bundestag, um über ein Thema zu diskutieren, das für unsere Gesellschaft von entscheidender Bedeutung ist: die Gesundheitspolitik in Deutschland.`

**Example span labels:**
`[1, 1, 1, 1, 1, 1, 1, 1, 1]`

**Example label:**
`1`

---



**Domain:** `student_essays`
**Agent:** `gpt_4o_mini_gemma2_9b`
**Feature Agent:** `gpt2_512`

**Train:** 50734
**Test:** 14496
**Eval:** 7248

**Example-Features:**
`[[4.4132913899375126e-05, 4.203895392974451e-45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00015184479707386345], [9.98157247522613e-06, 2.268475629563227e-09, 2.4706817147723825e-10, 2.598772308459729e-10, 4.608947598572222e-11, 2.177385494128714e-10, 7.711160811448015e-13, 6.366168985201537e-13, 2.281021511211531e-17, 9.533206545892253e-25, 7.987888483894126e-31, 0.0, 0.029878340661525726]]...`

**Feature-Shape:**
`(512, 13)`

**Example text:**
`[Your Name] [Your Address] [City, State, Zip Code] [Email Address] [Date] The Honorable [Senator's Name] [Senator's Office Address] [City, State, Zip Code] Dear Senator [Senator's Last Name], I hope this letter finds you well. I’m writing to you today as a concerned citizen who’s really worried about the way our presidential elections work. Honestly, I can’t wrap my head around the Electoral College anymore. It feels like an outdated system that just doesn’t represent us, the people. I mean, how is it fair that a candidate can win the presidency without winning the popular vote? It’s like saying my vote doesn’t count just because I live in a state that leans one way or another. That doesn’t seem right, does it? We should all have an equal say in who leads our country, and the popular vote is the only way to truly reflect the will of the people. The Electoral College creates a situation where some votes are more valuable than others, and that’s just plain wrong. It leads to candidates ignoring vast swathes of the country because they know they won’t win those states. Isn’t it time we changed that? We need a system that encourages candidates to engage with all of us, not just the folks in swing states. I urge you to consider advocating for a popular vote system. It’s time for our elections to truly represent the voice of the people. Thank you for your time, and I hope you’ll take my concerns to heart.`

**Example-Tokenized Text:**
`[[58], [7120], [6530], [60], [685], [7120], [17917], [60], [685], [14941]]...`

**Tokenized Text Shape:**
`(512, 1)`

**Sentence-Token-Spans:**
`[[0, 61], [61, 87], [87, 103], [103, 121], [121, 141], [141, 168], [168, 179], [179, 209], [209, 233], [233, 256], [256, 266], [266, 288], [288, 300], [300, 317], [317, 336]]`

**Example Sentence-Token Span decoded:**
`[Your Name] [Your Address] [City, State, Zip Code] [Email Address] [Date] The Honorable [Senator's Name] [Senator's Office Address] [City, State, Zip Code] Dear Senator [Senator's Last Name], I hope this letter finds you well.`

**Example span labels:**
`[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]`

**Example label:**
`1`

---


# Data Loaders

Create the data loaders for training and evaluation.

In [212]:
class LuminarSequenceDataset(Dataset):
    def __init__(self, dataset, feature_key="features"):
        self.samples = []
        for example in dataset:
            spans = example["sentence_token_spans"]
            features = torch.tensor(example[feature_key])  # (seq_len, feature_dim)
            labels = example["span_labels"]
            self.samples.append({
                "features": features,
                "sentence_spans": spans,
                "span_labels": labels
            })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


def collate_fn(batch):
    features = [item["features"] for item in batch]
    sentence_spans = [item["sentence_spans"] for item in batch]
    span_labels = [item["span_labels"] for item in batch]
    features = torch.stack(features)
    return {
        "features": features,
        "sentence_spans": sentence_spans,
        "span_labels": span_labels
    }

In [220]:
from torch.utils.data import ConcatDataset, DataLoader

train_datasets = []
test_loaders = []

all_train_parts = []
all_test_parts = []

for domain in domains:
    print(f"Creating datasets for domain: {domain}")

    # Merge train and eval for cross-validation
    train_dataset = LuminarSequenceDataset(
        ConcatDataset([
            datasets[domain]["gpt_4o_mini_gemma2_9b"]["gpt2_512"]["train"],
            datasets[domain]["gpt_4o_mini_gemma2_9b"]["gpt2_512"]["eval"]
        ])
    )
    print(f"Train Dataset: {len(train_dataset)}")
    train_datasets.append((domain, train_dataset))
    all_train_parts.append(train_dataset)

    # Prepare test loader
    test_dataset = LuminarSequenceDataset(
        datasets[domain]["gpt_4o_mini_gemma2_9b"]["gpt2_512"]["test"]
    )
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
    print(f"Test Dataset: {len(test_dataset)}")
    test_loaders.append((domain, test_loader))
    all_test_parts.append(test_dataset)

# Now concat all
all_domains_train_datasets = ConcatDataset(all_train_parts)
all_domains_test_dataset = ConcatDataset(all_test_parts)
all_domains_test_loader = DataLoader(all_domains_test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

print(f"All Train: {len(all_domains_train_datasets)}")
train_datasets.append(("all", all_domains_train_datasets))

print(f"All Test: {len(all_domains_test_dataset)}")
test_loaders.append(("all", all_domains_test_loader))


Creating datasets for domain: bundestag
Train Dataset: 16090
Test Dataset: 4024
Creating datasets for domain: student_essays
Train Dataset: 57982
Test Dataset: 14496
All Train: 74072
All Test: 18520


In [221]:
def evaluate_metrics(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            features = batch["features"].to(device)
            sentence_spans = batch["sentence_spans"]
            span_labels = batch["span_labels"]

            output = model(features, sentence_spans)
            probs = torch.sigmoid(output.logits).view(-1).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            labels = torch.cat([torch.tensor(lbl, dtype=torch.int) for lbl in span_labels]).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return acc, f1


def evaluate(model, eval_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in eval_loader:
            features = batch["features"].to(device)
            sentence_spans = batch["sentence_spans"]
            span_labels = batch["span_labels"]

            output = model(features, sentence_spans, span_labels=span_labels)
            loss = output.loss
            total_loss += loss.item()
    avg_loss = total_loss / len(eval_loader)
    return avg_loss


def train_and_evaluate(model, train_loader, eval_loader, optimizer, device, epochs, patience=3):
    best_eval_loss = float("inf")
    best_train_loss = None
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Training
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            features = batch["features"].to(device)
            sentence_spans = batch["sentence_spans"]
            span_labels = batch["span_labels"]

            optimizer.zero_grad()
            output = model(features, sentence_spans, span_labels=span_labels)
            loss = output.loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        # Evaluation
        avg_eval_loss = evaluate(model, eval_loader, device)
        print(f"Train Loss: {avg_train_loss:.4f} | Eval Loss: {avg_eval_loss:.4f}")

        # Early Stopping & Checkpoint
        if avg_eval_loss < best_eval_loss:
            best_eval_loss = avg_eval_loss
            best_train_loss = avg_train_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping after {epoch + 1} epochs.")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break

    print(f"\nBest Eval Loss: {best_eval_loss:.4f} | Best Train Loss: {best_train_loss:.4f}")
    return model

# Training

Train the model using K-Fold Cross Validation on the training dataset.

In [258]:
#from luminar.sequence_classifier import LuminarSequence
from src.luminar.sequence_classifier import LuminarSequence

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = {
    "feature_dim": (Config.FEATURE_LEN, Config.NUM_INTERMEDIATE_LIKELIHOODS),
    "feature_type": "intermediate_likelihoods",
    "feature_selection": "first",
    "conv_layer_shapes": (
        ConvolutionalLayerSpec(128, 5),
        ConvolutionalLayerSpec(256, 5),
        ConvolutionalLayerSpec(128, 3),
    ),
    "lstm_hidden_dim": 256,
    "lstm_layers": 1,
    "projection_dim": 128,
    "early_stopping_patience": 3,
    "learning_rate": 9e-5,
    "max_epochs": 40,
    "seed": Config.SEED,
    "rescale_features": False,
    "stack_spans": 1
}

In [259]:
def run_kfold_training(
        dataset, test_loader, model_config, num_folds=5, epochs=10, batch_size=32, patience=3, device="cpu"
):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_metrics = []

    indices = list(range(len(dataset)))
    best_models = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
        print(f"\n========== Fold {fold + 1}/{num_folds} ==========")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
        eval_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

        model = LuminarSequence(**model_config).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=model_config["learning_rate"])

        model = train_and_evaluate(model, train_loader, eval_loader, optimizer, device, epochs, patience)
        best_models.append((fold, model))

        # Evaluate after training on the validation set
        acc, f1 = evaluate_metrics(model, eval_loader, device)
        print(f"[Fold {fold + 1}] Final Accuracy: {acc:.4f} | F1: {f1:.4f}")
        fold_metrics.append({"accuracy": acc, "f1": f1})

    # Summarize results
    avg_acc = sum(m["accuracy"] for m in fold_metrics) / num_folds
    avg_f1 = sum(m["f1"] for m in fold_metrics) / num_folds
    print(f"\n========== K-Fold Cross Validation Results ==========")
    for i, m in enumerate(fold_metrics):
        print(f"Fold {i + 1}: Accuracy = {m['accuracy']:.4f}, F1 = {m['f1']:.4f}")
    print(f"\nAverage Accuracy: {avg_acc:.4f} | Average F1: {avg_f1:.4f}")
    print()

    print(f"\n========== Test Evaluation of all K-Fold Cross Model ==========")
    for (fold, model) in best_models:
        test_acc, test_f1 = evaluate_metrics(model, test_loader, device)
        print(f"K-Fold {fold}: Test Accuracy = {test_acc:.4f}, Test F1 = {test_f1:.4f}")


In [ ]:
for train_dataset, test_loader in zip(train_datasets, test_loaders):
    print(f"Training on domain: {train_dataset[0]}")

    # Reset the random seed for reproducibility
    torch.manual_seed(Config.SEED)
    np.random.seed(Config.SEED)

    # Unpack the datasets
    train_dataset = train_dataset[1]
    test_loader = test_loader[1]

    print(f"Train Dataset Size: {len(train_dataset)}")
    print(f"Test Dataset Size: {len(test_loader.dataset)}")

    # Run K-Fold training
    run_kfold_training(train_dataset, test_loader, config, num_folds=5, epochs=120, batch_size=512, patience=14, device=device)

Training on domain: bundestag
Train Dataset Size: 16090
Test Dataset Size: 4024

========== Fold 1/5 ==========
LuminarTrainingConfig(feature_dim=(512, 13), feature_type='intermediate_likelihoods', feature_selection='first', conv_layer_shapes=((128, 5, 1), (256, 5, 1), (128, 3, 1)), lstm_hidden_dim=256, lstm_layers=1, projection_dim=128, early_stopping_patience=3, learning_rate=9e-05, max_epochs=40, seed=42, rescale_features=False, stack_spans=1)

Epoch 1/120
